# Integration of lakeFS with Dagster

## Versioning Information

In [ ]:
sourceBranch = "main"

## Import Python packages

In [ ]:
import datetime
import os

## Set environment variables

In [ ]:
os.environ["LAKEFS_ENDPOINT"] = lakefsEndPoint
os.environ["LAKEFS_CREDENTIALS_ACCESS_KEY_ID"] = lakefsAccessKey
os.environ["LAKEFS_CREDENTIALS_SECRET_ACCESS_KEY"] = lakefsSecretKey

## Working with the lakeFS Python client API

###### Note: To learn more about lakeFS Python integration visit https://docs.lakefs.io/integrations/python.html

In [ ]:
%xmode Minimal
if not 'lakefs' in locals():
    import lakefs_client
    from lakefs_client.models import *
    from lakefs_client.client import LakeFSClient

    # lakeFS credentials and endpoint
    configuration = lakefs_client.Configuration()
    configuration.username = lakefsAccessKey
    configuration.password = lakefsSecretKey
    configuration.host = lakefsEndPoint

    lakefs = LakeFSClient(configuration)
    print("Created lakeFS client.")

#### Verify lakeFS credentials by getting lakeFS version

In [ ]:
print("Verifying lakeFS credentials…")
try:
    v=lakefs.config.get_lake_fs_version()
except:
    print("🛑 failed to get lakeFS version")
else:
    print(f"…✅lakeFS credentials verified\n\nℹ️lakeFS version {v.version}")

## Create Repository

In [ ]:
from lakefs_client.exceptions import NotFoundException

try:
    repo=lakefs.repositories.get_repository(repo_name)
    print(f"Found existing repo {repo.id} using storage namespace {repo.storage_namespace}")
except NotFoundException as f:
    print(f"Repository {repo_name} does not exist, so going to try and create it now.")
    try:
        repo=lakefs.repositories.create_repository(repository_creation=RepositoryCreation(name=repo_name,
                                                                                                storage_namespace=f"{storageNamespace}"))
        print(f"Created new repo {repo.id} using storage namespace {repo.storage_namespace}")
    except lakefs_client.ApiException as e:
        print(f"Error creating repo {repo_name}. Error is {e}")
        os._exit(00)
except lakefs_client.ApiException as e:
    print(f"Error getting repo {repo_name}: {e}")
    os._exit(00)

## Start Dagit UI

In [ ]:
%%script bash --bg --out script_out --err script_error
dagit -h 0.0.0.0 -p 3000
echo "Dagit UI is running"